In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
ipl2020=pd.read_csv(r'C:\Users\Yogesh Arora\Downloads\IPL Ball-by-Ball 2008-2020\IPL Ball-by-Ball 2008-2020.csv')
ipl2021=pd.read_csv(r'C:\Users\Yogesh Arora\Downloads\archive\IPL Ball-by-Ball Dataset 2021.csv')

In [3]:
ipl2020.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [4]:
ipl2021.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,1254058,1,0,1,RG Sharma,CA Lynn,Mohammed Siraj,2,0,2,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
1,1254058,1,0,2,RG Sharma,CA Lynn,Mohammed Siraj,0,0,0,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
2,1254058,1,0,3,RG Sharma,CA Lynn,Mohammed Siraj,0,0,0,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
3,1254058,1,0,4,RG Sharma,CA Lynn,Mohammed Siraj,2,0,2,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
4,1254058,1,0,5,RG Sharma,CA Lynn,Mohammed Siraj,0,0,0,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore


In [5]:
matchdata = pd.concat(
    map(pd.read_csv, [r'C:\Users\Yogesh Arora\Downloads\IPL Ball-by-Ball 2008-2020\IPL Ball-by-Ball 2008-2020.csv', r'C:\Users\Yogesh Arora\Downloads\archive\IPL Ball-by-Ball Dataset 2021.csv']), ignore_index=True)

In [6]:
match_data = pd.read_csv(r'C:\Users\Yogesh Arora\Downloads\IPL Matches 2008-2020.csv')

In [7]:
match_data["season"] = pd.to_datetime(match_data["date"]).dt.year

In [8]:
matchdata = matchdata.merge(match_data[["id", "season"]], on="id", how="inner")

In [9]:
matchdata.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,season
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008


In [10]:
matchdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193468 entries, 0 to 193467
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                193468 non-null  int64 
 1   inning            193468 non-null  int64 
 2   over              193468 non-null  int64 
 3   ball              193468 non-null  int64 
 4   batsman           193468 non-null  object
 5   non_striker       193468 non-null  object
 6   bowler            193468 non-null  object
 7   batsman_runs      193468 non-null  int64 
 8   extra_runs        193468 non-null  int64 
 9   total_runs        193468 non-null  int64 
 10  non_boundary      193468 non-null  int64 
 11  is_wicket         193468 non-null  int64 
 12  dismissal_kind    9495 non-null    object
 13  player_dismissed  9495 non-null    object
 14  fielder           6784 non-null    object
 15  extras_type       10233 non-null   object
 16  batting_team      193468 non-null  obj

# Aggregated Batsman Data with Match Id

In [11]:
        batsmen_score = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
        batsmen_scores_30 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_30"})
        batsmen_scores_50 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=50).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_50"})
        batsmen_scores_100 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=100).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_100"})
        batsmen_scores_0 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())==0).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_0"})
        batsmen_ball_faced = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['over'].count()).\
            rename(columns={"over": "total_balls_faced"})
        batsmen_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'batsman'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_faced"})
        batsmen_scores6 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_6"})
        batsmen_scores4 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_4"})
        batsmen_position = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].min())
       
        
        df_list = [batsmen_score, batsmen_ball_faced, batsmen_ball_faced_legal, batsmen_scores6, batsmen_scores4, batsmen_position,batsmen_scores_30,batsmen_scores_50,batsmen_scores_100,batsmen_scores_0]
        batsmen_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        batsmen_summary = pd.merge(batsmen_summary, matchdata[['id', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                                   drop_duplicates(), on=['id', 'batsman'], how='left')
        batsmen_summary.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'Playing Team',
                                        'bowling_team': 'Opponent Team'}, inplace=True)
       

In [12]:
batsmen_summary

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,total_runs_100,total_runs_0,batsmen_innings,Playing Team,Opponent Team
0,335982,AA Noffke,9,12,11.0,NaN,1.0,0,0,0,0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
1,335982,B Akhil,0,2,2.0,NaN,NaN,0,0,0,0,1,2,Royal Challengers Bangalore,Kolkata Knight Riders
2,335982,BB McCullum,158,77,73.0,13.0,10.0,0,1,1,1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,CL White,6,10,8.0,NaN,NaN,0,0,0,0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
4,335982,DJ Hussey,12,12,12.0,NaN,1.0,0,0,0,0,0,1,Kolkata Knight Riders,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12157,1237181,RR Pant,56,40,38.0,2.0,4.0,0,1,1,0,0,1,Delhi Capitals,Mumbai Indians
12158,1237181,S Dhawan,15,13,13.0,NaN,3.0,0,0,0,0,0,1,Delhi Capitals,Mumbai Indians
12159,1237181,SA Yadav,19,20,20.0,1.0,1.0,0,0,0,0,0,2,Mumbai Indians,Delhi Capitals
12160,1237181,SO Hetmyer,5,5,5.0,NaN,1.0,0,0,0,0,0,1,Delhi Capitals,Mumbai Indians


# Bowler Data Aggregated with Match Id

In [13]:
        bowler_wickets = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "total_wickets"})
        bowler_overs_bow = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].count()).\
            rename(columns={"over": "total_balls_bowled"})
        bowler_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'bowler'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_bowled"})
        # calculating the number of maiden overs bowled in a match
        bowler_runs_given = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs_given"})
        bowler_wickets_3 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==3).astype(int).\
            rename(columns={"is_wicket": "total_wickets_3"})
        bowler_wickets_4 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==4).astype(int).\
            rename(columns={"is_wicket": "total_wickets_4"})
        bowler_wickets_5 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==5).astype(int).\
            rename(columns={"is_wicket": "total_wickets_5"})
        bowler_maiden = pd.DataFrame(matchdata.groupby(['id', 'bowler', 'over'])['total_runs'].sum()).reset_index()
        bowler_maiden['maiden_overs'] = np.where(bowler_maiden['total_runs'] == 0, 1, 0)
        bowler_maiden = bowler_maiden.groupby(['id', 'bowler'])['maiden_overs'].sum()
        
        #legbyes to be considered
        
        df_list = [bowler_wickets, bowler_overs_bow, bowler_ball_faced_legal, bowler_runs_given,bowler_wickets_3,bowler_wickets_4,bowler_wickets_5,bowler_maiden]
        bowler_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        # adding additional columns just in case we need for modeling
        bowler_summary = pd.merge(bowler_summary, matchdata[['id', 'bowler', 'inning', 'batting_team', 'bowling_team']].drop_duplicates(), on=['id', 'bowler'], how='left')
        bowler_summary.rename(columns={'inning': 'bowlers_innings', 'bowler': 'playername', 'batting_team': 'Playing team',
                                       'bowling_team': 'Opponent Team'}, inplace=True)
        bowler_summary['economy_rate'] = (bowler_summary['total_runs_given'] * 6) / bowler_summary['total_legal_balls_bowled']
        bowler_summary['eco less than 5'] = (bowler_summary['economy_rate']<=5).astype(int)
        bowler_summary['eco 5-5.99'] = ((bowler_summary['economy_rate']<6)&(bowler_summary['economy_rate']>=5)).astype(int)
        bowler_summary['eco 6-7'] = ((bowler_summary['economy_rate']>=6)&(bowler_summary['economy_rate']<7)).astype(int)
        bowler_summary['eco 7-10'] = ((bowler_summary['economy_rate']>=7)&(bowler_summary['economy_rate']<10)).astype(int)
        bowler_summary['eco 11-12'] = ((bowler_summary['economy_rate']>=11)&(bowler_summary['economy_rate']<12)).astype(int)
        bowler_summary['eco greater than 12'] = (bowler_summary['economy_rate']>12).astype(int)
        
        
        

In [14]:
bowler_summary

,id,playername,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,total_wickets_3,total_wickets_4,total_wickets_5,maiden_overs,bowlers_innings,Playing team,Opponent Team,economy_rate,eco less than 5,eco 5-5.99,eco 6-7,eco 7-10,eco 11-12,eco greater than 12
0,335982,AA Noffke,1.0,25,24,35,0.0,0.0,0.0,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,8.750000,0,0,0,1,0,0
1,335982,AB Agarkar,3.0,28,24,21,1.0,0.0,0.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,5.250000,0,1,0,0,0,0
2,335982,AB Dinda,2.0,20,18,7,0.0,0.0,0.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2.333333,1,0,0,0,0,0
3,335982,CL White,NaN,7,6,22,NaN,NaN,NaN,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,22.000000,0,0,0,0,0,1
4,335982,I Sharma,1.0,19,18,6,0.0,0.0,0.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2.000000,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9662,1237181,MP Stoinis,1.0,12,12,23,0.0,0.0,0.0,0,2,Mumbai Indians,Delhi Capitals,11.500000,0,0,0,0,1,0
9663,1237181,NM Coulter-Nile,2.0,25,24,28,0.0,0.0,0.0,0,1,Delhi Capitals,Mumbai Indians,7.000000,0,0,0,1,0,0
9664,1237181,P Dubey,NaN,18,18,29,NaN,NaN,NaN,0,2,Mumbai Indians,Delhi Capitals,9.666667,0,0,0,1,0,0
9665,1237181,R Ashwin,NaN,24,24,28,NaN,NaN,NaN,0,2,Mumbai Indians,Delhi Capitals,7.000000,0,0,0,1,0,0


In [ ]:
fielder_data = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (matchdata['dismissal_kind'].isin(['caught','caught and bowled','run out','stumped'])))].
                                      groupby(['id','fielder','dismissal_kind'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "Fielding_stats"})

# Batsman,Bowler Merged Statistics

In [15]:
  ipl_merged_scorecard = pd.merge(batsmen_summary, bowler_summary, on=['id', 'playername'], how='outer')

In [ ]:
for col in list_of_cols:
  #ipl_merged_scorecard.loc[ ipl_merged_scorecard[col].isnull() , col ] = 0
  ipl_merged_scorecard[col].fillna(0, inplace=True)
ipl_merged_scorecard.isnull().sum()

# Batsman, Bowler, All Rounder Classification

In [16]:
        MINAVGBALLSFACED = 8
        MINAVGBOWLSBOWLED = 6
        player_avg = ipl_merged_scorecard[['playername', 'total_balls_faced', 'total_balls_bowled']].fillna(0)
        player_avg = pd.DataFrame(player_avg.groupby('playername')[['total_balls_faced', 'total_balls_bowled']].mean())
        conditions = [((player_avg['total_balls_faced'] >= MINAVGBALLSFACED) & (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)),
                      (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)]
        choices = ['AllRounder', 'Bowler']
        player_avg['playing_role'] = np.select(conditions, choices, default='Batsmen')
        player_avg = player_avg.reset_index()

In [17]:
  ipl_merged_scorecard = pd.merge(ipl_merged_scorecard, player_avg[['playername', 'playing_role']], on='playername', how='left')

In [18]:
ipl_merged_scorecard

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing team,Opponent Team_y,economy_rate,eco less than 5,eco 5-5.99,eco 6-7,eco 7-10,eco 11-12,eco greater than 12,playing_role
0,335982,AA Noffke,9.0,12.0,11.0,NaN,1.0,0.0,0.0,0.0,...,Kolkata Knight Riders,Royal Challengers Bangalore,8.750000,0.0,0.0,0.0,1.0,0.0,0.0,AllRounder
1,335982,B Akhil,0.0,2.0,2.0,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bowler
2,335982,BB McCullum,158.0,77.0,73.0,13.0,10.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batsmen
3,335982,CL White,6.0,10.0,8.0,NaN,NaN,0.0,0.0,0.0,...,Kolkata Knight Riders,Royal Challengers Bangalore,22.000000,0.0,0.0,0.0,0.0,0.0,1.0,Batsmen
4,335982,DJ Hussey,12.0,12.0,12.0,NaN,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batsmen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17146,1237181,K Rabada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mumbai Indians,Delhi Capitals,10.666667,0.0,0.0,0.0,0.0,0.0,0.0,Bowler
17147,1237181,NM Coulter-Nile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Delhi Capitals,Mumbai Indians,7.000000,0.0,0.0,0.0,1.0,0.0,0.0,Bowler
17148,1237181,P Dubey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mumbai Indians,Delhi Capitals,9.666667,0.0,0.0,0.0,1.0,0.0,0.0,Bowler
17149,1237181,R Ashwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mumbai Indians,Delhi Capitals,7.000000,0.0,0.0,0.0,1.0,0.0,0.0,Bowler


In [21]:
for col in list_of_cols:
  #ipl_merged_scorecard.loc[ ipl_merged_scorecard[col].isnull() , col ] = 0
  ipl_merged_scorecard[col].fillna(0, inplace=True)
ipl_merged_scorecard.isnull().sum()

NameError: name 'list_of_cols' is not defined

In [20]:
!pip install pulp

In [22]:
import pulp as p

# Aggregation through Season 

In [ ]:
        batsmen_score_s = pd.DataFrame(matchdata.groupby(['season', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
        batsmen_scores_30_s = pd.DataFrame(((matchdata.groupby(['season', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_30"})
        batsmen_scores_50_s = pd.DataFrame(((matchdata.groupby(['season', 'batsman'])['batsman_runs'].sum())>=50).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_50"})
        batsmen_scores_100_s = pd.DataFrame(((matchdata.groupby(['season', 'batsman'])['batsman_runs'].sum())>=100).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_100"})
        batsmen_scores_0_s = pd.DataFrame(((matchdata.groupby(['season', 'batsman'])['batsman_runs'].sum())==0).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_0"})
        batsmen_ball_faced_s = pd.DataFrame(matchdata.groupby(['season', 'batsman'])['over'].count()).\
            rename(columns={"over": "total_balls_faced"})
        batsmen_ball_faced_legal_s = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['season', 'batsman'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_faced"})
        batsmen_scores6_s = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['season', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_6"})
        batsmen_scores4_s = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['season', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_4"})
        batsmen_position_s = pd.DataFrame(matchdata.groupby(['season', 'batsman'])['is_wicket'].min())
       
        
        df_list_season = [batsmen_score_s, batsmen_ball_faced_s, batsmen_ball_faced_legal_s, batsmen_scores6_s, batsmen_scores4_s, batsmen_position_s,batsmen_scores_30_s,batsmen_scores_50_s,batsmen_scores_100_s,batsmen_scores_0_s]
        batsmen_summary_season = pd.concat(df_list_season, join='outer', axis=1).fillna(np.nan).reset_index()
        batsmen_summary_season = pd.merge(batsmen_summary_season, matchdata[['season', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                                   drop_duplicates(), on=['season', 'batsman'], how='left')
        batsmen_summary_season.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'Playing Team',
                                        'bowling_team': 'Opponent Team'}, inplace=True)

In [ ]:
 batsmen_summary_season